<h1 style = "font-size: 30px; text-align: center;">AI Genetic Hands On</h1>
<h2 style = "font-size: 25px; text-align: center;">Hospital Job Scheduling</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Name: Daneshvar Amrollahi</h2>
<h2 style = "font-size: 25px; text-align: center; color: #666">Student Id: 810197685</h2>
<h4 style="text-align: center">Spring 1400</h4>

<h2 style = "font-size: 25px;">Notes</h2>

<h3>Tests: </h3>

<p style="text-indent :2em;"><b>Test1: </b><mark>Average time: 0.41s < 15s</mark></p>
<p style="text-indent :2em;"><b>Test2: </b><mark>Average time: 8.73 < 60s</mark></p>
    

<h3>Outputs: </h3>
<p style="text-indent :2em;">Respectively <mark>output1.txt</mark> and <mark>output2.txt</mark></p>

<br>

<h2 style = "font-size: 25px;">Test Files</h2>

In [72]:
testFile1 = "test1.txt"
testFile2 = "test2.txt"

outFile1 = "output1.txt"
outFile2 = "output2.txt"

<h2 style = "font-size: 25px;">Reading Test File Content</h2>

In [73]:
def readInput(testFile) :
    file = open(testFile, 'r+')
    fileList = file.readlines()
    fileList = [s.replace('\n', '') for s in fileList]
    
    [days, doctors] = [int(i) for i in fileList[0].split()]
    maxCapacity = int(fileList[1])
    
    allShifts = []
    for i in range(2, days + 2):
        dayRequirements = fileList[i].split()
        morningReqs = [int(i) for i in dayRequirements[0].split(",")]
        eveningReqs = [int(i) for i in dayRequirements[1].split(",")]
        nightReqs = [int(i) for i in dayRequirements[2].split(",")]
        
        allShifts.append((morningReqs, eveningReqs, nightReqs))

    file.close()
    return [days, list(range(doctors)), maxCapacity, allShifts]

<h2 style = "font-size: 25px;">Job Scheduler</h2>

In [74]:
from random import choice
import random
from math import floor

class JobScheduler:
    def __init__(self, fileInfo):
        self.days = fileInfo[0]
        self.doctors = len(fileInfo[1])
        self.doctorsIds = fileInfo[1]
        self.maxCapacity = fileInfo[2]
        self.allShifts = fileInfo[3]
        
        

        # You can use these values for your genetic algorithm
        self.popSize = 300
        self.crossOverPoints = (self.doctors // 5)
        self.elitismPercentage = 16 #(move x% best of parents directly to the new population)
        self.pc = 0.65 #(crossover probability)
        self.pm = 0.05  #(mutation probability)
        
        self.chromosomes = self.generateInitialPopulation()
        
        self.goalReached = 0
        self.ans = []
        
    def generateInitialPopulation(self):
        initPop = []
        for t in range(self.popSize):
            chromosome = []
            for i in range(self.doctors):
                row = []
                for j in range(self.days):
                    row.append(choice(['m', 'e', 'n', 'o'])) #morning, evening, night, off
                chromosome.append(row)
            
                
            #for i in range(self.doctors):
            #    for j in range(self.days):
            #        print(chromosome[i][j], end = " ")
            #    print()
            
            #print()
            
            initPop.append(chromosome)
        
        
        return initPop
        
    
    def crossOver(self, chromosome1, chromosome2):
            
        col = [0] * self.days #col[i] = X: pick that column from parX
        for t in range(self.crossOverPoints):
            colNum = random.randint(0, self.days - 1)
            prob = random.random()
            if (prob < self.pc):
                col[colNum] = 1 - col[colNum]

        child = []
        for i in range(self.doctors):
            row = []
            for j in range(self.days):
                if (col[j] == 0):
                    row.append(chromosome1[i][j])
                else:
                    row.append(chromosome2[i][j])

            child.append(row)
            
        return child
        
                
    def mutate(self, chromosome):
        for i in range(self.doctors):
            for j in range(self.days):
                prob = random.random()
                if (prob < self.pm):
                    chromosome[i][j] = choice(['m', 'e', 'n', 'o'])
        
        return chromosome
        
        
    def calculateCost(self, chromosome):
        c1 = 0 #number of violations on capacities for each doctor

        
        for i in range(self.doctors):
            morning = chromosome[i].count('m')
            evening = chromosome[i].count('e')
            night = chromosome[i].count('n')
            if (morning + evening + night > self.maxCapacity):
                c1 += 1
        
        c2 = 0 #min-max violations on each shift
        for j in range(self.days):
            morning = 0
            evening = 0
            night = 0
            for i in range(self.doctors):
                morning += (chromosome[i][j] == 'm')
                evening += (chromosome[i][j] == 'e')
                night += (chromosome[i][j] == 'n')
            
            if (not (self.allShifts[j][0][0] <= morning and morning <= self.allShifts[j][0][1]) ):
                c2 += 1
                
            if (not (self.allShifts[j][1][0] <= evening and evening <= self.allShifts[j][1][1]) ):
                c2 += 1
                
            if (not (self.allShifts[j][2][0] <= night and night <= self.allShifts[j][2][1]) ):
                c2 += 1
        
        c3 = 0 #violations on shifts
        for i in range(self.doctors):
            for j in range(self.days):
                if (j >= 1):
                    c3 += (chromosome[i][j] == 'm' and chromosome[i][j - 1] == 'n')
                    c3 += (chromosome[i][j] == 'e' and chromosome[i][j - 1] == 'n')
                if (j >= 2):
                    c3 += (chromosome[i][j] == 'n' and chromosome[i][j - 1] == 'n' and chromosome[i][j - 2] == 'n')
        
        
        
        w1, w2, w3 = 1, 1, 1
        
        return w1 * c1 + w2 * c2 + w3 * c3
    
    
    def generateNewPopulation(self):
        curPop = self.chromosomes
        
        curChromosomes = []
        for chromosome in self.chromosomes:
            #print("current chromosome:")
            #print(chromosome)
            curChromosomes.append((self.calculateCost(chromosome), chromosome))
        
        curChromosomes.sort() #based on cost
        
        cost = curChromosomes[0][0]
        #print(cost)
        
        goodChromosomes = curChromosomes[:floor((self.elitismPercentage / 100) * (self.popSize))]
        if (curChromosomes[0][0] == 0): #no violations
            self.goalReached = 1
            self.ans = curChromosomes[0][1]
            return
        
        
        
        newPop = [chromosome[1] for chromosome in goodChromosomes]
        
        while (len(newPop) < self.popSize):
            par0, par1 = random.sample(goodChromosomes, 2)
            child = self.crossOver(par0[1], par1[1])
            mutatedChild = self.mutate(child)
            
            newPop.append(mutatedChild)
            
        return newPop
    
    
    def schedule(self): 
        while (not(self.goalReached)):
            newPop = self.generateNewPopulation()
            self.chromosomes = newPop
        return self.ans

A **Genetic Algorithm** is used in order to solve this problem.

A **chromosome** in this problem is a $p \times d$ matrix where **p** is the number of doctors and **d** is the number of days. The value in the $i$ th row and $j$th column is chose from the set $\{m, e, n, o\}$ indicating respectively morning, evening, night, and off showing the shift for the $i$th doctor in the $j$th day.

Each column in the matrix above is considered as a **gene**.

A **crossover** is applied using the following method: Two parents $p_1$ and $p_2$ are selected. There is also a parameter called $\textit{crossOverPoints}$. The child is by default supposed to be exactly as same as $p_1$ but for crossOverPoints times, a random column is chose and with the probability $p_{mutation}$ this column will be chose from the other parent. 

A **mutation** is applied by randomly changing a value of a cell in the matrix to something from the set $\{m, e, n, o\}$

The **fittness function** is sort of implemented reversly as a **cost function**. The lesser value it has, the more close it is to the answer. 
The number of violations on each rule is computed separately:

$c_{1} = $ Number of doctors with working shifts more than their capacity 

$c_{2} = $ Number of working shifts that are not in their desired interval $\{min, max\}$

$c_{3} = $ Number of violations on working shifts of a doctor: 
    
    * 2 consecutive shifts of n,m and n,e
    * 3 consecutive shifts of n,n,n
    
The cost function is implemented as $w_1c_1 + w_2c_2 + w_3c_3$ where here $w_1=w_2=w_3=1$
    


<h2 style = "font-size: 25px;">Output Generation</h2>

In [75]:
def printWithComma(file, a):
    if (len(a) == 0):
        file.write("empty ")
        print("empty ", end = "")
        return
    
    for i in range(len(a) - 1):
        print(str(a[i]), end = ",")
        file.write(str(a[i]))
        file.write(",")
    
    #print("len is", len(a))
    print(str(a[-1]), end = " ")
    file.write(str(a[-1]))
    file.write(" ")

def writeOutput(fileName):
    file = open(fileName, "w")
    

    for j in range(len(ans[0])): #days
        morning = []
        evening = []
        night = []
        for i in range(len(ans)):
            if (ans[i][j] == 'm'):
                morning.append(i)
            if (ans[i][j] == 'e'):
                evening.append(i)
            if (ans[i][j] == 'n'):
                night.append(i)

        #print("morn")
        #print(morning)
        printWithComma(file, morning)
        #print("even")
        printWithComma(file, evening)
        #print("nigh")
        printWithComma(file, night)
        file.write("\n")
        print()
        
    file.close()


<h2 style = "font-size: 25px;">Execution</h2>

In [76]:
import time

fileInfo1 = readInput(testFile1)

start = time.time()

scheduler = JobScheduler(fileInfo1)
ans = scheduler.schedule()

end = time.time()

print("test 1: ", '%.2f'%(end - start), 'sec')

for i in range(len(ans)):
    for j in range(len(ans[0])):
        print(ans[i][j], end = " ")
    print()
print()    

writeOutput(outFile1)
print()

test 1:  0.59 sec
o m o e m e o e o o 
n o e n n o e m o n 
o o m o o n o n o o 
m e o n o m n o e e 
n n o n o o m e e n 
e o n o o n o n n o 
m e e m m o m o o m 

3,6 5 1,4 
0 3,6 4 
2 1,6 5 
6 0 1,3,4 
0,6 empty 1 
3 0 2,5 
4,6 1 3 
1 0,4 2,5 
empty 3,4 5 
6 3 1,4 



In [77]:
fileInfo2 = readInput(testFile2)

start = time.time()

scheduler = JobScheduler(fileInfo2)
ans = scheduler.schedule()

end = time.time()

print("Test 2: ", '%.2f'%(end - start), 'sec')

for i in range(len(ans)):
    for j in range(len(ans[0])):
        print(ans[i][j], end = " ")
    print()
print()

writeOutput(outFile2)
print()

Test 2:  5.12 sec
e e m e e n o e e e o o o o o 
m m o m m n o n n o o n o n o 
m o n o o e o e e n o m o m e 
e m e e e o n n o m o o m e n 
m o o m n o m e n n o o n o e 
e n o o o m n o m e e o m o o 
n o o n o m e m e o m m n n o 
o m n o o o o e e m o e e e m 
e n o n o e o m o o m o e n o 

1,2,4 0,3,5,8 6 
1,3,7 0 5,8 
0 3 2,7 
1,4 0,3 6,8 
1 0,3 4 
5,6 2,8 0,1 
4 6 3,5 
6,8 0,2,4,7 1,3 
5 0,2,6,7 1,4 
3,7 0,5 2,4 
6,8 5 empty 
2,6 7 1 
3,5 7,8 4,6 
2 3,7 1,6,8 
7 2,4 3 

